# Import software libraries

In [ ]:
import sys                                                  # Read system parameters.
import numpy as np                                          # Work with multi-dimensional arrays.
import pandas as pd                                         # Manipulate and analyze data.
import sklearn                                              # Train and evaluate machine learning models.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, \
                                  MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from collections import Counter                             # Count objects in containers.
import warnings
warnings.filterwarnings('ignore')

# Ensure results are reproducible.
np.random.seed(1)

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- NumPy {}'.format(np.__version__))
print('- pandas {}'.format(pd.__version__))
print('- scikit-learn {}'.format(sklearn.__version__))

# Load and preview the data

In [ ]:
users_data = pd.read_pickle('/home/student/CDSP/Finalizing/data/users_data_final.pickle')

users_data.head(n = 3)

# Split the data into train and test sets

In [ ]:
target = users_data.term_deposit
features = users_data.drop(['user_id', 'term_deposit'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(features, target)

# Define an initial pipeline

In [ ]:
pipe = Pipeline([('scaler', MinMaxScaler()),
                 ('reduce_dim', PCA(n_components = 2)),
                 ('model', DecisionTreeClassifier())])

# Evaluate the initial pipeline

In [ ]:
pipe = pipe.fit(X_train, y_train)
print('Model accuracy on test data:', pipe.score(X_test, y_test))

In [ ]:
y_pred = pipe.predict(X_test)
print(Counter(y_pred))

In [ ]:
results = pd.concat([y_test.iloc[:5], X_test.iloc[:5]], axis = 1)
results.insert(1, 'term_deposit_pred', y_pred[:5])
results

# Tune the pipeline

In [ ]:
scalers = [None, StandardScaler(), MinMaxScaler()]
pca_dims = [None, PCA(n_components = 2), PCA(n_components = 5)]
models = [DecisionTreeClassifier(),
          RandomForestClassifier(random_state = 1)]

params = {'scaler': scalers,
          'reduce_dim': pca_dims,
          'model': models}

In [ ]:
gs = GridSearchCV(pipe, params, n_jobs = -1, verbose = 2,
                  cv = StratifiedKFold(5, shuffle = False)). \
fit(X_train, y_train)

# Evaluate the tuned pipeline

In [ ]:
print('Best accuracy score:', gs.score(X_test, y_test))
print('Best parameters:    ', gs.best_params_)

In [ ]:
y_pred = gs.predict(X_test) 
print(Counter(y_pred))

In [ ]:
results['term_deposit_pred'] = y_pred[:5]
results

# Test the model generated by the pipeline on new data

In [ ]:
new_data = pd.read_csv('/home/student/CDSP/Finalizing/data/new_users_data.csv')

new_data

In [ ]:
y_pred = gs.predict(new_data)

new_data.insert(0, 'term_deposit_pred', y_pred)
new_data